## Intial analysis of water data collected in Alicante

In [1]:
import warnings
# warnings.filterwarnings('ignore')
import os
import sys
import math
import pandas as pandas
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from statistics import mean


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from modules import water_monitoring
from modules.water_monitoring import WaterMonitoringInstance
from modules.data_analyzer import plot_real_and_predicted, Analyzer

## Reading all the data

In [2]:
instance_13 = WaterMonitoringInstance()
instance_14 = WaterMonitoringInstance()
instance_15 = WaterMonitoringInstance()
instance_16 = WaterMonitoringInstance()
instance_17 = WaterMonitoringInstance()
instance_18 = WaterMonitoringInstance()
instance_19 = WaterMonitoringInstance()

instance_13.read_data("../data/Benalua Consumption/Benalua 2013.csv", ";")
instance_14.read_data("../data/Benalua Consumption/Benalua 2014.csv", ";")
instance_15.read_data("../data/Benalua Consumption/Benalua 2015.csv", ";")
instance_16.read_data("../data/Benalua Consumption/Benalua 2016.csv", ";")
instance_17.read_data("../data/Benalua Consumption/Benalua 2017.csv", ";")
instance_18.read_data("../data/Benalua Consumption/Benalua 2018.csv", ";")
instance_19.read_data("../data/Benalua Consumption/Benalua 2019.csv", ";")
instance_19.get_data_frame()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.



,CA009-52,Unnamed: 1,CT005-52,Unnamed: 3,Unnamed: 4
0,31/12/2018 14:03:05,"49,55154",31/12/2018 14:15:00,"9163241,54",NaN
1,31/12/2018 14:03:36,"57,871",31/12/2018 14:30:00,"9163255,24",NaN
2,31/12/2018 14:04:07,"56,1334",31/12/2018 14:45:00,"9163268,64",NaN
3,31/12/2018 14:04:37,"62,02201",31/12/2018 15:00:00,"9163282,24",NaN
4,31/12/2018 14:05:08,"53,10789",31/12/2018 15:15:00,"9163296,24",NaN
...,...,...,...,...,...
987914,16/12/2019 13:20:57,"62,88913",NaN,NaN,NaN
987915,16/12/2019 13:21:28,"58,23195",NaN,NaN,NaN
987916,16/12/2019 13:21:58,"59,27594",NaN,NaN,NaN
987917,16/12/2019 13:22:29,"59,87085",NaN,NaN,NaN


In [3]:
instance_13.transform_data("alicante", True)
instance_14.transform_data("alicante", True)
instance_15.transform_data("alicante", True)
instance_16.transform_data("alicante", True)
instance_17.transform_data("alicante", True)
instance_18.transform_data("alicante", True)
instance_19.transform_data("alicante", True)

instance_13.make_time_features("CA009-52")
instance_14.make_time_features("CA009-52")
instance_15.make_time_features("CA009-52")
instance_16.make_time_features("CA009-52")
instance_17.make_time_features("CA009-52")
instance_18.make_time_features("CA009-52")
instance_19.make_time_features("CA009-52")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dataframe_arr = [instance_13.get_data_frame(), instance_14.get_data_frame(), instance_15.get_data_frame(),
                 instance_16.get_data_frame(), instance_17.get_data_frame(), instance_18.get_data_frame()]

analyzer = Analyzer(instance_19.get_data_frame())
analyzer.random_forest_train(dataframe_arr, ["Minute", "Hour", "Day", "Month", "Year"], "Data-1(Sensor1)", "CA009-52")

y_test, y_pred, time = analyzer.random_forest_test(["Minute", "Hour", "Day", "Month", "Year"], "Data-1(Sensor1)", "CA009-52")

plot_real_and_predicted(y_test, y_pred, time)

             CA009-52  Data-1(Sensor1)  Minute  Hour  Day  Month  Year
0 2013-01-02 03:25:36         25.98000      25     3    2      1  2013
1 2013-03-13 10:32:12         78.76678      32    10   13      3  2013
2 2013-03-13 10:32:41         78.84520      32    10   13      3  2013
             CA009-52  Data-1(Sensor1)  Minute  Hour  Day  Month  Year
0 2013-12-31 15:13:58         78.45468      13    15   31     12  2013
1 2013-12-31 15:14:29         75.73385      14    15   31     12  2013
2 2013-12-31 15:15:00         74.26326      15    15   31     12  2013
             CA009-52  Data-1(Sensor1)  Minute  Hour  Day  Month  Year
0 2014-12-31 15:13:54         59.59104      13    15   31     12  2014
1 2014-12-31 15:14:24         61.49341      14    15   31     12  2014
2 2014-12-31 15:14:55         67.38917      14    15   31     12  2014
             CA009-52  Data-1(Sensor1)  Minute  Hour  Day  Month  Year
0 2016-01-01 15:13:56         45.72545      13    15    1      1  2016
1 2016

In [ ]:
instance_17.plot_two_columns("CA009-52", "Data-1(Sensor1)")

In [ ]:
arr = [df.set_index(["CA009-52"]) for df in dataframe_arr]
merged_data_frame = pandas.concat(arr, axis=0, sort=True).reset_index()
merged_data_frame

In [ ]:
merged_data_frame.to_csv("Benalua_all_year_2013-2018.csv", sep=';', encoding='utf-8')

In [ ]:
instance = WaterMonitoringInstance()
hourly_based = merged_data_frame.resample('H', on="CA009-52").mean().reset_index()
instance.set_data_frame(hourly_based)
# instance.interactive_two_columns_plot("CA009-52", "Data-1(Sensor1)")
hourly_based

In [ ]:
instance_v2 = WaterMonitoringInstance()
instance_v2.read_data("../data/Benalua Consumption/Benalua 2015.csv", ";")
instance_v2.transform_data("alicante", True)
instance_v2.make_time_features("CA009-52")

In [ ]:
"""data_v2 = instance_v2.get_data_frame()
analyzer_2 = Analyzer(data_v2)
y_test_2, y_pred_2, time_2 = analyzer_2.random_forest(["Minute", "Hour", "Day", "Month", "Year"], "Data-1(Sensor1)", "CA009-52")
plot_real_and_predicted(y_test_2, y_pred_2, time_2)""";

In [ ]:
"""instance_2 = water_monitoring.WaterMonitoringInstance()
instance_2.read_data("../data/water_loss_data_set_1.json", "json")
instance_2.get_data_frame()"""

In [ ]:
"""instance_2.transform_data("braila", True)
data_2 = instance_2.get_data_frame()
data_2"""

In [ ]:
"""
subset_data = instance.take_data_in_range("2017-05-13 10:30:00", "2017-07-13 10:30:00", "CA009-52")
subset_data

coef_arr = instance.calculate_correlation("Data-1(Sensor1)") # instance.get_nan_rows()
coef_arr 

instance.plot_two_columns("CA009-52", "Data-1(Sensor1)")

# instance.interactive_two_columns_plot("CA009-52", "Data-1(Sensor1)")

instance.transform_data("alicante", True)
instance.make_time_features("CA009-52")

data = instance.get_data_frame()
data

analyzer = Analyzer(data)
y_test, y_pred, time = analyzer.random_forest(["Hour"], "Data-1(Sensor1)", "CA009-52")
plot_real_and_predicted(y_test, y_pred, time)

"""